# Data acquisition in arbok

### Arbok module imports

In [1]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.adb import ADB
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

2023-09-18 11:40:57,257 - qm - INFO     - Starting session: 0bfb4d89-d57b-49d8-9d6c-47de7114a759


In [2]:
from arbok.core.program import Program
from arbok.core.sample import Sample

Instanciating sample and measurement objects

In [3]:
aurora = Sample('aurora', aurora_qua_config)
ESR_search = Program('ESR_search', sample = aurora)

In [4]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.adb import ADB
from arbok.samples.aurora.control.rabi import Rabi
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

Creating Subsequences and adding them to the measurement object `ESR_search`

In [5]:
init6 = InitOdd6dot('init6')
adb6 = ADB('adb6')
read6 = PSB6dotRead('read6')

In [6]:
ESR_search.add_subsequence(init6)
ESR_search.add_subsequence(adb6)
ESR_search.add_subsequence(read6)

In [7]:
ESR_search.adb6.chirp_rate()

21000

# Running the program in the quantify framework

In [8]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl
from quantify_core.visualization.pyqt_plotmon import PlotMonitor_pyqt

Specifying the path for the database containing results

In [9]:
dh.set_datadir()

Data will be saved in:
C:\Users\labuser\quantify-data


Next we connect the physical OPX+ instrument to our program ... 

In [10]:
ESR_search.connect_OPX(host_ip= '192.168.0.3')

2023-09-18 11:40:58,938 - qm - INFO     - Performing health check
2023-09-18 11:40:58,960 - qm - INFO     - Health check passed
2023-09-18 11:41:01,584 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing existing identical Quantum Machine qm-1695001863604
2023-09-18 11:41:01,586 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1695001863604 is canceling running job 1684735462167.


Instanciating `MeasurementControl`

In [11]:
meas_ctrl = MeasurementControl('meas_ctrl')

Creating and connecting a `PlotMonitor` to our `MeasurementControl`

In [12]:
plotmon = PlotMonitor_pyqt("plotmon")
meas_ctrl.instr_plotmon(plotmon.name)

C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots is deprecated. Use "qcodes_loop.plots" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots.colors is deprecated. Use "qcodes_loop.plots.colors" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\qcodes\plots\pyqtgraph.py:17: QCoDeSDeprecationWarning: The qcodes.utils.qt_helpers module is deprecated. Use "qcodes_loop.utils.qt_helpers" as an alternative.
  import qcodes.utils.qt_helpers
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:19: QCoDeSDeprecationWarning: The qcodes.plots.pyqtgraph is deprecated. Use "qcodes_loop.plots

## How to measure: adiabatic ESR search

So far so good, now fully set up our adiabatic ESR search. We have a init, control and readout sequence. Now we have to define parameters to sweep and to measure.

Defining the frequency sweep list:

In [13]:
import numpy as np

In [14]:
f_detune=np.arange(25e6,80e6,0.1e6, dtype = int)

Defining the parameters to be swept and adding setpoints to them. Also parameters to be recorded are added. In this case the measured state of SET 1 and 2

In [15]:
ESR_search.settables = [
    ESR_search.adb6.IfQ1]
ESR_search.setpoints_grid = [
    f_detune]
ESR_search.gettables = [
    ESR_search.read6.diff_set2_state]

Just defined values are automatically added to `meas_ctrl` by running `prepare_meas_ctrl`

In [16]:
ESR_search.prepare_meas_ctrl(meas_ctrl)

... and run the job! Dont worry, the program will only run the first iteration and wait for us to retreive the result and only continue after that.

In [17]:
ESR_search.run()

Adding qua <class 'int'> variable for IfQ1
Adding qua sweep loop for IfQ1
2023-09-18 11:41:04,472 - qm - INFO     - Sending program to QOP for compilation
2023-09-18 11:41:36,242 - qm - INFO     - Executing program


If all this is done, the actual measurement can be run! Check the taskbar for your plot monitor and see the buffered results come in.

In [18]:
#meas_ctrl.run("adb_ESR_search", soft_avg= 20)

## How to measure: Rabi Chevron

Sometimes we want to measure two dimensional parameter sweeps. This works very similarly to the 1D case. We just have to add a second settable a corresponding value array to the setpoints.

But lets first create the sequence:

In [19]:
RabiChevron = Program('RabiChevron', sample = aurora)
rabi6 = Rabi('rabi6')
RabiChevron.add_subsequence(init6)
RabiChevron.add_subsequence(rabi6)
RabiChevron.add_subsequence(read6)

In [20]:
f_detune=np.arange(32.8e6,34.2e6,0.05e6, dtype = int)
print(len(f_detune))
t_list = np.arange(4,30e3/4,200, dtype = int)
print(len(t_list))

28
38


In [21]:
RabiChevron.settables = [
    RabiChevron.rabi6.IfQ1,
    RabiChevron.rabi6.tRabi]
RabiChevron.setpoints_grid = [
    f_detune,
    t_list]
RabiChevron.gettables = [
    RabiChevron.read6.diff_set1_state,
    RabiChevron.read6.diff_set2_state]

RabiChevron.connect_OPX(host_ip= '192.168.0.3')
RabiChevron.prepare_meas_ctrl(meas_ctrl)

2023-09-18 11:41:36,648 - qm - INFO     - Performing health check
2023-09-18 11:41:36,716 - qm - INFO     - Health check passed
2023-09-18 11:41:40,755 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing existing identical Quantum Machine qm-1695001956295
2023-09-18 11:41:40,756 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1695001956295 is canceling running job 1684735462168.


In [22]:
RabiChevron.run()

Adding qua <class 'int'> variable for IfQ1
Adding qua <class 'int'> variable for tRabi
Adding qua sweep loop for tRabi
Adding qua sweep loop for IfQ1
2023-09-18 11:41:41,619 - qm - INFO     - Sending program to QOP for compilation
2023-09-18 11:41:50,917 - qm - INFO     - Executing program


In [23]:
meas_ctrl.run("RabiChevron", soft_avg=100)

Starting batched measurement...
Iterative settable(s) [outer loop(s)]:
	 --- (None) --- 
Batched settable(s):
	 IfQ1, tRabi 
Batch size limit: 1064

FETC 380 1064 diff_set1_state
WAIT 380 1064
WAIT 738 1064
WAIT 807 1064
READ 1064 1064
2023-09-18 11:41:52,531 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1064,)
FETC 1064 1064 diff_set2_state
READ 1064 1064
2023-09-18 11:41:53,549 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1064,)
  1% completed | elapsed time:      2s | time left:    287s  last batch size:   1064  FETC 1320 2128 diff_set1_state
WAIT 1320 2128
WAIT 1679 2128
WAIT 1821 2128
WAIT 1821 2128
READ 2128 2128
2023-09-18 11:41:55,816 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1064,)
FETC 2128 2128 diff_set2_state
READ 2128 2128
2023-09-18 11:41:56,700 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1064,)



KeyboardInterrupt

